# Chronos Bolt Base - Fine Tuning, With Covariates

Document: https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-chronos.html#fine-tuning , https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-chronos.html#incorporating-the-covariates

**The predictions of the covariate regressor are subtracted from the target column, and the univariate model then forecasts the residuals.**

In [1]:
import datetime

import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from sklearn.metrics import root_mean_squared_error

In [2]:
# Experiment設定
EXPERIMENT_NAME = 'ChronosBolt_FT_Covariate'
MODEL = "bolt_base"
FORECAST_HORIZON = 6

In [3]:
df_train = pd.read_csv('../data/walmart_ts_6_fcst_grp_train.csv')
df_test = pd.read_csv('../data/walmart_ts_6_fcst_grp_test.csv')
print(df_train.shape, df_test.shape)
#df_train.head()

# 日付カラムを型変換する場合
#df_train['Date'] = pd.to_datetime(df_train['Date'])
#df_test['Date'] = pd.to_datetime(df_test['Date'])

# 単一のグループIDを作成
df_train['Store_Dept'] = [str(i)+'_'+str(j) for i,j in zip(df_train['Store'], df_train['Dept'])]
df_test['Store_Dept'] = [str(i)+'_'+str(j) for i,j in zip(df_test['Store'], df_test['Dept'])]

# 利用カラムのみを選択
df_train = df_train[['Store_Dept','Date','Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday']]
df_test = df_test[['Store_Dept','Date','Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday']]
print(df_train.shape, df_test.shape)
df_train.head()

(702, 11) (36, 11)
(702, 9) (36, 9)


,Store_Dept,Date,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,IsHoliday
0,4_4,2010/2/5,59554.57,-1.0,-1.0,-1.0,-1.0,-1.0,0
1,4_4,2010/2/12,54069.82,-1.0,-1.0,-1.0,-1.0,-1.0,1
2,4_4,2010/2/19,53939.17,-1.0,-1.0,-1.0,-1.0,-1.0,0
3,4_4,2010/2/26,54687.08,-1.0,-1.0,-1.0,-1.0,-1.0,0
4,4_4,2010/3/5,56959.02,-1.0,-1.0,-1.0,-1.0,-1.0,0


In [4]:
# TimeSeriesDataFrameへ変換
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    id_column='Store_Dept',
    timestamp_column="Date"
)
print(type(train_data))
train_data.head()

<class 'autogluon.timeseries.dataset.ts_dataframe.TimeSeriesDataFrame'>


Weekly_Sales  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
item_id timestamp                                                              
4_4     2010-02-05      59554.57       -1.0       -1.0       -1.0       -1.0   
        2010-02-12      54069.82       -1.0       -1.0       -1.0       -1.0   
        2010-02-19      53939.17       -1.0       -1.0       -1.0       -1.0   
        2010-02-26      54687.08       -1.0       -1.0       -1.0       -1.0   
        2010-03-05      56959.02       -1.0       -1.0       -1.0       -1.0   

                    MarkDown5  IsHoliday  
item_id timestamp                         
4_4     2010-02-05       -1.0          0  
        2010-02-12       -1.0          1  
        2010-02-19       -1.0          0  
        2010-02-26       -1.0          0  
        2010-03-05       -1.0          0

In [5]:
# Test Dataも同様に変換。Covariatesを利用する場合、予測時に必要
test_data = TimeSeriesDataFrame.from_data_frame(
    df_test,
    #id_column=['Store', 'Dept'],
    id_column='Store_Dept',
    timestamp_column="Date"
)
test_data.head()

Weekly_Sales  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
item_id timestamp                                                              
4_4     2012-05-04      60576.41   13737.91       -1.0      93.65    6993.97   
        2012-05-11      57583.94   20499.88       86.0     103.05    5005.96   
        2012-05-18      60832.91    6531.12       -1.0      99.91    2290.18   
        2012-05-25      57781.87    4330.02      436.0     862.31    3268.41   
        2012-06-01      62758.52   10165.22        8.6      47.93    2676.33   

                    MarkDown5  IsHoliday  
item_id timestamp                         
4_4     2012-05-04    4541.89          0  
        2012-05-11    3815.16          0  
        2012-05-18    2521.84          0  
        2012-05-25    3749.53          0  
        2012-06-01    4896.34          0

In [6]:
%%time

log_dir = EXPERIMENT_NAME +'_'+ datetime.datetime.now().strftime('%Y-%m-%d_%Hh%Mm%Ss')
print(log_dir)

predictor = TimeSeriesPredictor(
    target="Weekly_Sales",
    eval_metric="RMSE",
    prediction_length=FORECAST_HORIZON,
    path="./tmp/"+log_dir,
    
    known_covariates_names=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday'],
).fit(
    train_data,
    hyperparameters={
        "Chronos": [
            {
                "model_path": MODEL,
                #"covariate_regressor": "XGB",  # XGBoost  -> カーネルが落ちてしまう、、。
                "covariate_regressor": "CAT",  # CatBoost
                "target_scaler": "standard",
                "fine_tune": True, 
                "ag_args": {"name_suffix": "WithRegressorFineTune"},
            },
        ],
    },
    enable_ensemble=False,
    time_limit=300,
)

Beginning AutoGluon training... Time limit = 300s
AutoGluon will save models to '/Users/YShimada-MBP16/Documents/Code_Github/DL_for_ImageData_and_Finetuning/TimeSeries_WM_simple/code/tmp/ChronosBolt_FT_Covariate_2025-03-01_09h30m12s'


ChronosBolt_FT_Covariate_2025-03-01_09h30m12s


=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.14
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:22:00 PST 2025; root:xnu-11215.81.4~3/RELEASE_X86_64
CPU Count:          16
GPU Count:          0
Memory Avail:       14.69 GB / 32.00 GB (45.9%)
Disk Space Avail:   1433.37 GB / 1863.39 GB (76.9%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': RMSE,
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'WithRegressorFineTune'},
                                  'covariate_regressor': 'CAT',
                                  'fine_tune': True,
                                  'model_path': 'bolt_base',
                                  'target_scaler': 'standard'}]},
 'known_covariates_names': ['MarkDown1',
                            'MarkDown2',
                            'MarkDown3',
                            'MarkDown4',
           

CPU times: user 30min 15s, sys: 1min 11s, total: 31min 27s
Wall time: 4min 6s


In [7]:
# 与えられたデータの最後の日付から、prediction_length分を予測
predictions = predictor.predict(train_data, known_covariates=test_data)
predictions

Model not specified in predict, will default to the model with the best validation score: ChronosWithRegressorFineTune[bolt_base]


mean           0.1           0.2           0.3  \
item_id timestamp                                                            
4_4     2012-05-04  59152.718287  56471.482585  57271.558167  57895.686465   
        2012-05-11  58873.999054  56497.656957  57215.401746  57777.648725   
        2012-05-18  57412.750681  55116.920965  55794.526758  56313.151495   
        2012-05-25  63599.213428  61293.593423  61987.697181  62526.309561   
        2012-06-01  59732.952663  57246.229303  58017.928034  58606.509781   
        2012-06-08  60252.791710  58059.956836  58691.175709  59230.879486   
8_13    2012-05-04  24686.872299  23486.027188  23869.493796  24163.179085   
        2012-05-11  24807.994437  23724.528404  24081.551990  24346.123083   
        2012-05-18  25201.053032  24121.615552  24486.298050  24740.246652   
        2012-05-25  25718.269674  24624.508729  25008.112679  25271.741173   
        2012-06-01  24782.995741  23634.812593  24028.902264  24310.630752   
        2012-06-08  25659.048463  24598.730992  24954.811020  25225.992242   
1_13    2012-05-04  42584.779835  40544.440005  41225.950811  41699.596442   
        2012-05-11  41631.764440  39848.050933  40458.537789  40882.403844   
        2012-05-18  40696.645280  39098.345559  39636.550397  39980.840431   
        2012-05-25  42412.498729  40674.939271  41270.026065  41674.716271   
        2012-06-01  42146.898618  39960.986884  40717.569318  41232.166660   
        2012-06-08  43319.534200  41246.471499  41948.394382  42446.371488   
13_8    2012-05-04  35612.868610  34170.607746  34627.198096  34975.493185   
        2012-05-11  34644.272763  33412.707410  33809.031706  34104.126508   
        2012-05-18  33529.456326  32362.845937  32777.724822  33052.161568   
        2012-05-25  34949.193951  33629.458428  34096.255356  34409.143588   
        2012-06-01  37140.376690  35243.204089  35977.819739  36432.898059   
        2012-06-08  37881.615772  36143.439945  36778.243059  37186.929742   
19_8    2012-05-04  43057.678691  41089.705752  41737.855646  42218.670456   
        2012-05-11  41203.303864  39272.745779  39914.380338  40393.788734   
        2012-05-18  41182.486582  39270.446965  39911.096373  40374.652291   
        2012-05-25  44195.220310  42307.268790  42944.498727  43405.607567   
        2012-06-01  40966.655692  39069.624700  39709.280323  40174.146765   
        2012-06-08  42815.405056  40908.265727  41556.415425  42031.264214   
31_13   2012-05-04  42941.865719  41629.458774  42043.132649  42370.486860   
        2012-05-11  42953.294275  41617.957363  42066.718237  42411.894310   
        2012-05-18  45846.540165  44167.850586  44770.402097  45157.887723   
        2012-05-25  45529.699970  44074.303345  44595.965673  44930.347488   
        2012-06-01  43514.804897  42240.257241  42692.952383  43001.096194   
        2012-06-08  44008.221081  42747.176586  43186.557674  43530.614689   

                             0.4           0.5           0.6           0.7  \
item_id timestamp                                                            
4_4     2012-05-04  58508.532184  59152.718287  59857.064029  60644.172511   
        2012-05-11  58317.919899  58873.999054  59463.817719  60163.575435   
        2012-05-18  56888.794727  57412.750681  57956.816669  58565.534567   
        2012-05-25  63062.704326  63599.213428  64146.528922  64768.941388   
        2012-06-01  59180.778372  59732.952663  60289.319999  60922.937218   
        2012-06-08  59724.929593  60252.791710  60797.451990  61431.096494   
8_13    2012-05-04  24429.038265  24686.872299  24955.169460  25251.800719   
        2012-05-11  24583.042972  24807.994437  25034.295929  25309.613968   
        2012-05-18  24991.758154  25201.053032  25410.374280  25642.880190   
        2012-05-25  25505.054277  25718.269674  25929.710790  26173.910680   
        2012-06-01  24556.640144  24782.995741  25000.346360  25250.051838   
        2012-06-08  25442.884675  25659.048463  2

In [8]:
df_pred = predictions.to_data_frame().reset_index()

print('RMSE: ', root_mean_squared_error(y_true=df_test['Weekly_Sales'], y_pred=df_pred['mean']))

df_pred.to_csv('../Chronos_results/{}.csv'.format(EXPERIMENT_NAME), index=False)

RMSE:  1960.9385327866046
